In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import json
import struct
from collections import defaultdict

In [3]:
data_primary_raw = pd.read_csv("data.csv")
data_secondary_raw = pd.read_csv("data_secondary.csv")

datatypes = {7: 4, 1: 1}
point_step = 64
tags_raw = (
    {b"name": "x", b"datatype": 7, b"count": 1, b"offset": 0},
    {b"name": "y", b"datatype": 7, b"count": 1, b"offset": 4},
    {b"name": "z", b"datatype": 7, b"count": 1, b"offset": 8},
    {b"name": "obj_vrel_long", b"datatype": 7, b"count": 1, b"offset": 12},
    {b"name": "obj_lat_speed", b"datatype": 7, b"count": 1, b"offset": 16},
    {b"name": "obj_rcs_value", b"datatype": 7, b"count": 1, b"offset": 20},
    {b"name": "radial_speed_absolute", b"datatype": 7, b"count": 1, b"offset": 24},
    {b"name": "distance_accuracy", b"datatype": 7, b"count": 1, b"offset": 28},
    {b"name": "angle_accuracy", b"datatype": 7, b"count": 1, b"offset": 32},
    {b"name": "pdh0", b"datatype": 7, b"count": 1, b"offset": 36},
    {b"name": "dist_long_rms", b"datatype": 7, b"count": 1, b"offset": 40},
    {b"name": "dist_lat_rms", b"datatype": 7, b"count": 1, b"offset": 44},
    {b"name": "v_long_rms", b"datatype": 7, b"count": 1, b"offset": 48},
    {b"name": "v_lat_rms", b"datatype": 7, b"count": 1, b"offset": 52},
    {b"name": "dyn_prop", b"datatype": 1, b"count": 1, b"offset": 56},
    {b"name": "range", b"datatype": 1, b"count": 1, b"offset": 57},
    {b"name": "has_quality", b"datatype": 1, b"count": 1, b"offset": 58},
    {b"name": "invalid", b"datatype": 1, b"count": 1, b"offset": 60},
    {b"name": "ambig", b"datatype": 1, b"count": 1, b"offset": 59},
)
tags = {i[b"name"]: (datatypes[i[b"datatype"]], i[b"offset"]) for i in tags_raw}

parsed_keys = (
    ("ride_date", "rover", "message_ts", "log_time", "ride_time")
    + tuple("primary_" + tag for tag in tags.keys())
    + tuple("secondary_" + tag for tag in tags.keys())
)
preff = ("primary_", "secondary_")
# data_secondary_raw

In [4]:
parsed = defaultdict(list)
for data_N, data in enumerate(
    (
        data_primary_raw["b'radar_primary_list'"],
        data_secondary_raw["b'radar_secondary_list'"],
    )
):
    for i, date_i in enumerate(data):
        dots_raw = bytes(map(int, date_i[1:-1].split(", ")))

        for k in ("ride_date", "rover", "message_ts", "log_time", "ride_time"):
            parsed[k].extend(
                [
                    (data_primary_raw if not data_N else data_secondary_raw)[
                        str(k.encode())
                    ][i]
                ]
                * (len(dots_raw) // 64)
            )

        for j in range(len(dots_raw) // 64):

            for tag_key, (tag_len, tag_offs) in tags.items():
                match tag_len:
                    case 4:
                        parsed[tag_key].append(
                            struct.unpack(
                                "f",
                                dots_raw[
                                    j * 64 + tag_offs : j * 64 + tag_offs + tag_len
                                ],
                            )[0]
                        )
                    case 1:
                        parsed[tag_key].append(
                            dots_raw[
                                j * 64 + tag_offs : j * 64 + tag_offs + tag_len
                            ].decode()
                        )

In [8]:
parsed_df = pd.DataFrame(parsed)
parsed_df.to_csv('clean_data.csv', index=False)
parsed_df

,ride_date,rover,message_ts,log_time,ride_time,x,y,z,obj_vrel_long,obj_lat_speed,...,pdh0,dist_long_rms,dist_lat_rms,v_long_rms,v_lat_rms,dyn_prop,range,has_quality,invalid,ambig
0,b'2025-01-23',b'matar',1737591862924763665,b'03:21:12',b'01:18:21',17.000000,1.100000,0.0,0.000000,0.0,...,0.25,1.023,0.616,0.288,0.011,, ,, ,
1,b'2025-01-23',b'matar',1737591862924763665,b'03:21:12',b'01:18:21',25.200001,3.900000,0.0,0.000000,0.0,...,0.75,1.317,0.616,0.288,0.011,, ,,,
2,b'2025-01-23',b'matar',1737591862924763665,b'03:21:12',b'01:18:21',35.000000,-7.100000,0.0,0.000000,0.0,...,0.75,1.697,0.794,0.288,0.011,, ,,,
3,b'2025-01-23',b'matar',1737591862924763665,b'03:21:12',b'01:18:21',37.200001,2.900000,0.0,0.000000,0.0,...,0.75,1.317,0.616,0.288,0.011,, ,,,
4,b'2025-01-23',b'matar',1737591862924763665,b'03:21:12',b'01:18:21',40.000000,-2.100000,0.0,0.000000,0.0,...,0.75,1.023,0.616,0.288,0.011,, ,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3757,b'2025-01-23',b'matar',1737586882561500541,b'01:18:21',b'01:18:21',201.800003,26.299999,0.0,-23.798738,0.0,...,0.75,4.676,6.025,0.288,0.011,,,,,
3758,b'2025-01-23',b'matar',1737586882561500541,b'01:18:21',b'01:18:21',214.000000,8.700000,0.0,-23.980192,0.0,...,0.75,4.676,3.630,0.288,0.011,,,,,
3759,b'2025-01-23',b'matar',1737586882561500541,b'01:18:21',b'01:18:21',235.000000,17.500000,0.0,-23.933729,0.0,...,0.75,4.676,7.762,0.288,0.011,,,,,
3760,b'2025-01-23',b'matar',1737586882561500541,b'01:18:21',b'01:18:21',274.000000,23.299999,0.0,-24.162794,0.0,...,0.75,4.676,3.630,0.288,0.011,,,, ,
